# variances


那么如果`A <: B`，`C[A]`和`C[B]`的关系是怎样？

* `C[A] <: C[B]`，则`C`是`covariant class`
* `C[A] >: C[B]`，则`C`是`contravariant class`
* `C[A] C[B]互不为子类`，则`C`是`nonvariant class`

符号表示为
```scala
class C[+T] {...} // covariant class
class C[-T] {...} // contravariant class
class C[T] {...}  // nonvariant class
```


## List和Array

如果`A <: B`，那么`List(A) <: List(B)`，而`Array`是`nonvarianct class`

这是因为`List`是不可变量，`Array`是可变的

In [2]:
// 这样就会出问题，因为Array可变，所以这样操作后，类型会乱掉
// 所以第二行在编译时就会出问题
val a: Array[Int] = Array(1)
val b: Array[AnyVal] = a
b(0) = '1'
val s: Int = a(0)

cmd2.sc:2: type mismatch;
 found   : Array[Int]
 required: Array[AnyVal]
Note: Int <: AnyVal, but class Array is invariant in type T.
You may wish to investigate a wildcard type such as `_ <: AnyVal`. (SLS 3.2.10)
val b: Array[AnyVal] = a
                       ^

: 

In [2]:
// 而List不可变，所以编译通过了
val a: List[Int] = List(1)
val b: List[AnyVal] = a
// b(0) = '1'
// val s: AnyVal = a(0)

a: List[Int] = List(1)
b: List[AnyVal] = List(1)

## Function

函数的参数类型是`contravariant`，而函数的返回值是`covariant`

```scala
package scala
trait Function1[-T, +U] {
  def apply(x: T): U
}
```

原因的话，在<a href="./type_bounds.ipynb#type_rules_for_functions">type_bounds.ipynb</a>中有解释

如果`A1 >: A2`而且`B1 <: B2`，则`A1 => B1 <: A2 => B2`


另外，scala会对variance进行简单的检查

* covariant type parameter只能出现在方法返回值中
* contravariant type parameter只能出现在方法参数中
* nonvariant type paramter在哪都行

上面的Function1就通过了这个检查

## Example - List

**现在 有这样一个List**

In [6]:
trait List[+T] {
  def isEmpty: Boolean
  def head: T
  def tail: List[T]
}

class Cons[T](val head: T, val tail: List[T]) extends List[T] {
  def isEmpty = false
}

class Nil[T] extends List[T] {
  def isEmpty: Boolean = true
  def head: Nothing = throw new NoSuchElementException("Nil.head")
  def tail: Nothing = throw new NoSuchElementException("Nil.tail")
}

defined trait List
defined class Cons
defined class Nil

### 如果想要把Nil变成object该怎么写

In [5]:
trait List[+T] {
  def isEmpty: Boolean
  def head: T
  def tail: List[T]
}

class Cons[T](val head: T, val tail: List[T]) extends List[T] {
  def isEmpty = false
}

object Nil extends List[Nothing] { // 我们需要Nil是所有List[T]的子类，又因为+T，所以填Nothing
  def isEmpty: Boolean = true
  def head: Nothing = throw new NoSuchElementException("Nil.head")
  def tail: Nothing = throw new NoSuchElementException("Nil.tail")
}

defined trait List
defined class Cons
defined object Nil

下面这个调用可以通过编译

Nil是List[Nothing]，Nothing是String的子类，所以List[Nothing]是List[String]的子类

In [4]:
val x: List[String] = Nil

x: List[String] = $sess.cmd2Wrapper$Helper$Nil$@51a30db6

### 现在想在List中实现prepend

下面这种实现方式是不行的，因为参数不能是+T

举个例子，现在有`xs = List[IntSet]`，我能够`xs.prepend(Empty)`，因为`Empty`是`IntSet`的子类

又有`ys = List[NonEmpty]`，`ys.prepend(Emtpy)`是不对的，因为`Empty`不是是`NonEmpty`的子类

所以这时候违反了+T的原则（父类能干的事情子类都能干）

In [6]:
// 现在想在List中实现prepend
trait List[+T] {
  def isEmpty: Boolean
  def head: T
  def tail: List[T]
  def prepend(elem: T): List[T] = new Cons(elem, this) // 这样不行，因为参数不能是+T
}

class Cons[T](val head: T, val tail: List[T]) extends List[T] {
  def isEmpty = false
}

object Nil extends List[Nothing] {
  def isEmpty: Boolean = true
  def head: Nothing = throw new NoSuchElementException("Nil.head")
  def tail: Nothing = throw new NoSuchElementException("Nil.tail")
}


cmd6.sc:5: covariant type T occurs in contravariant position in type T of value elem
  def prepend(elem: T): List[T] = new Cons(elem, this)
              ^

: 

可以用下面的方式写，接受T的父类U

总结：

* covariant type parameter(+T)可能出现在方法参数的lower bound中(U >: T)
* contravariant type parameter(-T)可能出现在方法参数的upper bound中(U <: T)

In [7]:
// 现在想在List中实现prepend
trait List[+T] {
  def isEmpty: Boolean
  def head: T
  def tail: List[T]
  def prepend[U >: T](elem: U): List[U] = new Cons(elem, this)
}

class Cons[T](val head: T, val tail: List[T]) extends List[T] {
  def isEmpty = false
}

object Nil extends List[Nothing] {
  def isEmpty: Boolean = true
  def head: Nothing = throw new NoSuchElementException("Nil.head")
  def tail: Nothing = throw new NoSuchElementException("Nil.tail")
}


defined trait List
defined class Cons
defined object Nil

下面这个可以跑通（NonEmpty, Empty没在这里实现，有空加上吧）

原本xs是NonEmpty，调用了prepend后，接受了Empty类型的参数，scala会吧xs变为IntSet

In [7]:
def f(xs: List[NonEmpty], elem: Empty) = xs.prepend(x)

cmd7.sc:1: not found: type NonEmpty
def f(xs: List[NonEmpty], elem: Empty) = xs.prepend(x)
               ^cmd7.sc:1: not found: type Empty
def f(xs: List[NonEmpty], elem: Empty) = xs.prepend(x)
                                ^

: 